<img width="150" alt="Logo_ER10" src="https://user-images.githubusercontent.com/3244249/151994514-b584b984-a148-4ade-80ee-0f88b0aefa45.png">

# General overview of using dianna

DIANNA is a Python package that brings explainable AI (XAI) to your research project. It wraps carefully selected XAI methods (explainers) in a simple, uniform interface. It's built by, with and for (academic) researchers and research software engineers working on machine learning projects.

This overview illustrates the main strenghts of DIANNA, namely supporting many data modalities and several explainers. DIANNA is future-proof by supporting and advocating the ONNX de-facto standard for Neural Network models. Many modern frameworks alpready support native export to ONNX, for tutorials on conversion from PyTorch, Keras, Scikit-learn and TensorFlow see [conversion_onnx](https://github.com/dianna-ai/dianna/tree/main/tutorials/conversion_onnx) folder. 

## **General workflow**

1. Provide your *trained model* and *data item* ( *text, image, time series or tabular* )

```python
model_path = 'your_model.onnx'  # model trained on your data modality
data_item = <data_item> # data item for which the model's prediction needs to be explained 
```

2. If the task is classification: which are the *classes* your model has been trained for?

```python 
labels = [class_a, class_b]   # example of binary classification labels
```
*Which* of these classes do you want an explanation for?
```python
explained_class_index = labels.index(<explained_class>)  # explained_class can be any of the labels
```

3. Run dianna with the *explainer* of your choice ( *'LIME', 'RISE' or 'KernalSHAP'*) and visualize the output:

```python
explanation = dianna.<explanation_function>(model_path, data_item, explainer)
dianna.visualization.<visualization_function>(explanation[explained_class_index], data_item)
```

### Setting up

#### Colab Setup

In [2]:
running_in_colab = 'google.colab' in str(get_ipython())
if running_in_colab:
  # install dianna
  !python3 -m pip install dianna[notebooks]
  
  # download data used in this demo
  import os 
  base_url = 'https://raw.githubusercontent.com/dianna-ai/dianna/main/dianna/'
  paths_to_download = ['./data/binary-mnist.npz', './models/mnist_model_tf.onnx']
  for path in paths_to_download:
      !wget {base_url + path} -P {os.path.dirname(path)}
    
  paths_to_download = ['./data/movie_reviews_word_vectors.txt', './models/movie_review_model.onnx']
  for path in paths_to_download:
      !wget {base_url + path} -P {os.path.dirname(path)}    

  base_url = 'https://raw.githubusercontent.com/dianna-ai/dianna/main/dianna/'
  paths_to_download = ['./data/bee.jpg']
  for path in paths_to_download:
      !wget {base_url + path} -P {os.path.dirname(path)}

#### Libraries

In [ ]:
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore') # disable warnings relateds to versions of tf

import numpy as np

# for explanation and visualization
import dianna
from dianna import visualization
from dianna import utils
from dianna.utils.tokenizers import SpacyTokenizer

# ONNX
import onnx
import onnxruntime
from onnx_tf.backend import prepare

# text-related
import spacy
from torchtext.vocab import Vectors
from scipy.special import softmax
from scipy.special import expit as sigmoid

# keras model and preprocessing tools for Image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras import backend as K
from keras import utils

# visualizations
import matplotlib.pyplot as plt
%matplotlib inline

import random
random.seed(42)

## **Data modalities**
DIANNA supports text, images, time-series and tabular data.

### Text example* 

Lets illustrate the general worksflow above with *textual* data. The data item of interest is a sentence being (a part of) a movie review and the [model](https://zenodo.org/record/5910598) has been trained to classify the movie reviews from the [Stanford sentiment treebank](https://nlp.stanford.edu/sentiment/index.html) <img width="20" alt="nlp-logo_half_size" src="https://user-images.githubusercontent.com/3244249/152355020-908c04f3-aa99-489d-b87a-7e6b1f586118.png"> into 'positive' and 'negative' sentiment classes.
We are interested in which words are contributing positively (red) and which - negatively (blue) towards the model's decision to classify the review as positive and we would like to use the [LIME](https://www.kdd.org/kdd2016/papers/files/rfp0573-ribeiroA.pdf) explainer:

*for a full example see the /Explainers/LIME/lime_text tutorial

1. Provide your *trained model* and *text* of interest.

In [ ]:
# model paths and labels
model_path = Path('..','dianna','models', 'movie_review_model.onnx')
word_vector_path = Path('..','dianna','labels', 'movie_reviews_word_vectors.txt')
labels = ("negative", "positive")

The classifier accepts numerical tokens as input and outputs a score between 0 (the review is negative) and 1 (the review is positive).
Therefore, we define a model runner class, which accepts a sentence as input instead and returns one of two classes: negative or positive.

In [ ]:
# ensure the tokenizer for english is available
spacy.cli.download('en_core_web_sm')

In [ ]:
class MovieReviewsModelRunner:
    def __init__(self, model, word_vectors, max_filter_size):
        self.run_model = utils.get_function(str(model))
        self.vocab = Vectors(word_vectors, cache=os.path.dirname(word_vectors))
        self.max_filter_size = max_filter_size
        
        self.tokenizer =  SpacyTokenizer(name='en_core_web_sm')

    def __call__(self, sentences):
        # ensure the input has a batch axis
        if isinstance(sentences, str):
            sentences = [sentences]

        tokenized_sentences = []
        for sentence in sentences:
            # tokenize and pad to minimum length
            tokens = self.tokenizer.tokenize(sentence.lower())
            if len(tokens) < self.max_filter_size:
                tokens += ['<pad>'] * (self.max_filter_size - len(tokens))
            
            # numericalize the tokens
            tokens_numerical = [self.vocab.stoi[token] if token in self.vocab.stoi else self.vocab.stoi['<unk>']
                                for token in tokens]
            tokenized_sentences.append(tokens_numerical)
            
        # run the model, applying a sigmoid because the model outputs logits
        logits = self.run_model(tokenized_sentences)
        pred = np.apply_along_axis(sigmoid, 1, logits)
        
        # output two classes
        positivity = pred[:, 0]
        negativity = 1 - positivity
        return np.transpose([negativity, positivity])

In [ ]:
# define model runner. max_filter_size is a property of the model
model_runner = MovieReviewsModelRunner(model_path, word_vector_path, max_filter_size=5)

Define a sentence of interest:

In [ ]:
review = "A delectable and intriguing thriller filled with surprises"


   2. Which are the classes your model has been trained for? Which of these classes do you want an explanation for?

In [ ]:
labels = ("negative", "positive")  # sentiments of the movie reviews
explained_class_index = labels.index("positive")  # we are interested why our sentence is classified as having a positive sentiment
explained_class_index
labels.index('positive')


  3.  Run dianna with the explainer of your choice, 'LIME', and visualize the output. For textual data use the ```explain_text``` function.


In [ ]:
explanation = dianna.explain_text(model_runner, review, model_runner.tokenizer,'LIME', labels=[explained_class_index])[0]
explanation
fig, _ = visualization.highlight_text(explanation, model_runner.tokenizer.tokenize(review), colormap='bwr')

### Image example*

Here we apply the general workflow with image data from <img width="94" alt="ImageNet_autocrop" src="https://user-images.githubusercontent.com/3244249/152542090-fd78fde1-6dec-43b6-a7ae-eea964b8ae28.png">. The data item of interest is an image of a bee and we use the [ResNet 50](https://arxiv.org/abs/1512.03385) model trained on [Imagenet](https://image-net.org/download.php) to classify 1000 types of objects. We are interested in which pixels are contributing positively (red) and which - negatively (blue) towards the model's decision to classify the image as a 'bee' and we would like to use the [RISE](http://bmvc2018.org/contents/papers/1064.pdf) explainer:

*for a full example see the /Explainers/RISE/rise_imagenet tutorial

## **Explainers**
DIANNA supports [RISE](http://bmvc2018.org/contents/papers/1064.pdf), [LIME](https://www.kdd.org/kdd2016/papers/files/rfp0573-ribeiroA.pdf) and [KernalSHAP](https://proceedings.neurips.cc/paper/2017/file/8a20a8621978632d76c43dfd28b67767-Paper.pdf) explainability methods. This section demonstrates how to run all the supported explainers for the simple binary classification task of distinguishing the hand-written digits "0" and "1" on test example from the Binary MNIST dataset, a subset of the [MNIST benchmark](http://yann.lecun.com/exdb/mnist/). It also gives the basics for each of the explainers.

###  Explaining a Pretrained Binary MNIST Classification Model 

Load the Binary MNIST data, the pretrained [binary MNIST model](https://zenodo.org/record/5907177) and chose image to be explained.

In [ ]:
# load dataset
data_path = Path('..', 'dianna','data', 'binary-mnist.npz')
data = np.load(data_path)
# load testing data and the related labels
X_test = data['X_test'].astype(np.float32).reshape([-1, 28, 28, 1]) / 256
y_test = data['y_test']

In [ ]:
# Load saved onnx model
onnx_model_path = Path('..','dianna','models', 'mnist_model_tf.onnx')
onnx_model = onnx.load(onnx_model_path)
# get the output node
output_node = prepare(onnx_model, gen_tensor_dict=True).outputs[0]

Print class and image of a single instance in the test data for preview.

In [ ]:
# class name
class_name = ['digit 0', 'digit 1']
# instance index
i_instance = 3
# select instance for testing
test_sample = X_test[i_instance]
# model predictions with added batch axis to test sample
predictions = prepare(onnx_model).run(test_sample[None, ...])[f'{output_node}']
pred_class = class_name[np.argmax(predictions)]
# get the index of predictions
top_preds = np.argsort(-predictions)
inds = top_preds[0]
print("The predicted class for this test image is:", pred_class)
plt.imshow(X_test[i_instance][:,:,0], cmap='gray')  # 0 for channel

#### 1. KernelSHAP

SHapley Additive exPlanations, in short, SHAP, is a model-agnostic explainable AI approach which is used to decrypt the black-box models through estimating the [Shapley values](https://en.wikipedia.org/wiki/Shapley_value), which represent the relevancies of each data feature (image pixel, word in text, etc.). [KernelSHAP](https://proceedings.neurips.cc/paper/2017/file/8a20a8621978632d76c43dfd28b67767-Paper.pdf) is a variant of SHAP. It is a method that uses the LIME framework to compute Shapley Values, and visualizes the relevance attributions for each pixel/super-pixel by displaying them on an image. <br>

The user needs to specify the number model re-evaluations when explaining each prediction (`nsamples`). A binary mask need to be applied to the image indicating whihc image regiona are hidden. It requires the background color for the masked image, which can be specified by `background`.<br>

Performing KernelSHAP for each pixel is inefficient. It is always a good practice to segment the input image to super-pixels and perform computations on them. The user has to specify some keyword arguments related to the segmentation: the (approximate) number of labels in the segmented output image (`n_segments`), and width of Gaussian smoothing kernel for pre-processing for each image dimension (`sigma`).

In [ ]:
# use KernelSHAP to explain the network's predictions
shap_values = dianna.explain_image(onnx_model_path, test_sample,
                                  method="KernelSHAP", labels=[0], nsamples=1000,
                                  background=0, n_segments=200, sigma=0,
                                  axis_labels=('height','width','channels'))

Visualize the relevance scores of each pixel on the images.

In [ ]:
# Visualize the explanations
# get the range for color bar
max_val = np.max([np.max(np.abs(shap_values[i][:,:-1])) for i in range(len(shap_values))])
# plot the test image and the attributions on the image for class 0
print(f'Explaination for `{pred_class}` with KernelSHAP')

fig, _ = visualization.plot_image(shap_values[0], heatmap_cmap='bwr', heatmap_range=(-max_val, max_val))


The Shapley scores are estimated using KernelSHAP for models used to categorize the binary MNIST. The example here shows that the KernelSHAP method evaluates the importance of each segmentation/super pixel to the classification and the results are reasonably comparable to the human visual preception of the chosen testing hand-written digit image.

#### 2. RISE

RISE is short for Randomized Input Sampling for Explanation of Black-box Models. It estimates the relevance empirically by probing the model with randomly masked versions of the input image to obtain the corresponding outputs.<br>


RISE masks random portions of the input image and passes the masked image through the model — the portion that decreases the accuracy the most is the most “important” portion.<br>
To call the explainer and generate the relevance scores, the user need to specified the number of masks being randomly generated (`n_masks`), the resolution of features in masks (`feature_res`) and for each mask and each feature in the image, the probability of being kept unmasked (`p_keep`).

In [ ]:
relevances = dianna.explain_image(onnx_model_path, test_sample, method="RISE",
                                labels=[i for i in range(2)],
                                n_masks=5000, feature_res=8, p_keep=.1,
                                axis_labels=('height','width','channels'))[0]

Visualize the relevance scores for the predicted class on top of the image.

In [ ]:
print(f'Explaination for `{pred_class}` with RISE')
fig, _ = visualization.plot_image(relevances, X_test[i_instance][:,:,0], data_cmap='gray', heatmap_cmap='bwr')

#### 3. LIME

LIME (Local Interpretable Model-agnostic Explanations) is an explainable-AI method that aims to create an interpretable model that locally represents the classifier.


In [ ]:
# need to preprocess, because we divided the input data by 256 for the models and LIME needs RGB values
def preprocess_function(image):
    return (image / 256).astype(np.float32)

# An explanation is returned for each label, but we ask for just one label so the output is a list of length one.
explanation_heatmap = dianna.explain_image(onnx_model_path, test_sample * 256, 'LIME',
                                           axis_labels=('height','width','channels'),
                                           random_state=2,
                                           labels=[i for i in range(2)], preprocess_function=preprocess_function)

In [ ]:
print(f'Explaination for `{pred_class}` with LIME')
fig, _ = visualization.plot_image(explanation_heatmap[0], X_test[i_instance][:,:,0], heatmap_cmap='bwr')
